<link rel="stylesheet" href="/site-assets/css/gemma.css">
<link rel="stylesheet" href="https://fonts.googleapis.com/css2?family=Google+Symbols:opsz,wght,FILL,GRAD@20..48,100..700,0..1,-50..200" />

# Fine-tune Gemma models in Keras using LoRA

改編自 Google Cloud 官方範例：https://www.apache.org/licenses/LICENSE-2.0

## Overview

Gemma 是⼀系列輕量級的開放式 LLM 模型，建構於與打造 Gemini 模型相同的研發技術之上。

大型語言模型 (LLMs) 像 Gemma 已被證明能有效執行各種自然語言處理 (NLP) 任務。LLM 首先會透過自我監督的方式，在大量文本資料集上進行預訓練。預訓練可幫助 LLM 學習通用知識，例如詞語之間的統計關係。然後，LLM 可以使用特定領域的資料進行微調，以執行下游任務 (例如情緒分析)。

LLMs 的模型非常龐大。對於大多數應用程式來說，完整微調 (更新模型中的所有參數) 並非必要，因為典型的微調資料集通常遠遠小於預訓練資料集。

低秩適應 (LoRA) 是一種微調技術，可以大幅減少下游任務的可訓練參數量。該技術透過凍結模型權重並引入少量新權重的方式來實現這一目標。LoRA 微調速度更快、記憶體消耗更少，生成的模型權重更小 (幾百 MB)，同時還能維持模型輸出质量。

這個 Colab ，我將引導您使用 KerasNLP 執行 LoRA 微調，使用 Gemma 2B 模型以及 Databricks Dolly 15k 資料集 。此資料集包含 15,000 個由人類生成的高品質提示/回覆配對，專為微調 LLM 而設計。

## Setup

### 取得 Gemma 訪問權限

要完成本教學，您首先需要完成 [Gemma 設定](https://ai.google.dev/gemma/docs/setup) 中的設定指示。Gemma 設定指示將指導您完成以下步驟：

* 在 [kaggle.com](https://kaggle.com) 上獲得 Gemma 的訪問權限。
* 選擇一個具有足夠資源的 Colab 運行時環境，以運行 Gemma 2B 模型。
* 生成並配置您的 Kaggle 使用者名稱和 API 金鑰。

完成 Gemma 設定後，請進入下一個部分，為您的 Colab 環境設置環境變數。

### 選擇運行時環境

要完成本教學，您需要選擇一個具有足夠資源的 Colab 運行時環境來執行 Gemma 模型。在這裡，您可以使用 T4 GPU：

1. 在 Colab 視窗的右上角，選擇 &#9662;（**額外的連線選項**）。
2. 選擇 **變更運行時類型**。
3. 在 **硬體加速器** 下，選擇 **T4 GPU**。


### 配置您的 API 金鑰

要使用 Gemma，您必須提供您的 Kaggle 使用者名稱和 Kaggle API 金鑰。

要生成 Kaggle API 金鑰，請前往 Kaggle 使用者檔案的 **帳戶** 標籤頁並選擇 **Create New Token**。這將會下載一個包含您的 API 認證的 `kaggle.json` 檔案。

在 Colab 中，選擇左側面板中的 **Secrets**（🔑），然後新增您的 Kaggle 使用者名稱和 Kaggle API 金鑰。將您的使用者名稱儲存為 `KAGGLE_USERNAME`，並將您的 API 金鑰儲存為 `KAGGLE_KEY`。


### Set environment variables

Set environment variables for `KAGGLE_USERNAME` and `KAGGLE_KEY`.

In [1]:
import os
from google.colab import userdata

# Note: `userdata.get` is a Colab API. If you're not using Colab, set the env
# vars as appropriate for your system.

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

### Install dependencies

Install Keras, KerasNLP, and other dependencies.

In [2]:
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U "keras>=3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.2/572.2 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.5 MB/s eta 0:00:00


### Select a backend

Keras 是一個高階、多框架的深度學習 API，旨在簡化使用並提升使用便利性。使用 Keras 3，您可以在以下三種後端之一上運行工作流程：TensorFlow、JAX 或 PyTorch。

在本教學中，請將後端配置為 JAX。


In [3]:
os.environ["KERAS_BACKEND"] = "jax"  # Or "torch" or "tensorflow".
# Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Import packages

Import Keras and KerasNLP.

In [4]:
import keras
import keras_nlp

## Load Dataset

本次，我們使用的文件是：[erhwenkuo/dolly-15k-chinese-zhtw](https://huggingface.co/datasets/erhwenkuo/dolly-15k-chinese-zhtw)

In [5]:
!wget -O databricks-dolly-15k-zhtw.parquet https://huggingface.co/datasets/erhwenkuo/dolly-15k-chinese-zhtw/resolve/main/data/train-00000-of-00001-839cf763a52639ec.parquet

--2024-09-07 04:46:10--  https://huggingface.co/datasets/erhwenkuo/dolly-15k-chinese-zhtw/resolve/main/data/train-00000-of-00001-839cf763a52639ec.parquet
Resolving huggingface.co (huggingface.co)... 3.165.160.61, 3.165.160.12, 3.165.160.59, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.61|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/7f/a4/7fa4bedfecc28e6c287d997e1cf54c95a43f10cd85204e4590a9e456dfe93acf/54fcd90b0dfa518bac827f1016031be9a53684e9594d74afbd4b2e51b28c44ab?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27train-00000-of-00001-839cf763a52639ec.parquet%3B+filename%3D%22train-00000-of-00001-839cf763a52639ec.parquet%22%3B&Expires=1725943571&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyNTk0MzU3MX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy83Zi9hNC83ZmE0YmVkZmVjYzI4ZTZjMjg3ZDk5N2UxY2Y1NGM5NWE0M2YxMGNkODUyMDRlNDU5MGE5Z

預處理資料。本教學使用 1000 個訓練範例的子集來加快 notebook 的執行速度。若需更高品質的微調，請考慮使用更多的訓練資料。


In [6]:
import pandas as pd

# Read the Parquet file into a DataFrame
df = pd.read_parquet("databricks-dolly-15k-zhtw.parquet")

data = []

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    # Check if there is context, and skip if true
    if row["context"]:
        continue
    # Format the entire example as a single string
    template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
    data.append(template.format(instruction=row["instruction"], response=row["response"]))

# Only use 1000 training examples, to keep it fast
data = data[:1000]

## 載入模型

KerasNLP 提供了許多熱門[模型架構](https://keras.io/api/keras_nlp/models/)的實作。在本教學中，您將使用 `GemmaCausalLM` 建立一個模型，這是一個用於因果語言建模的端對端 Gemma 模型。因果語言模型會根據先前的標記預測下一個標記。

使用 `from_preset` 方法來建立模型：


In [7]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma2_2b_en")
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

`from_preset` 方法會從預設的架構和權重來實例化模型。在上述程式碼中，字串 "gemma2_2b_en" 指定了預設的架構，即擁有 20 億個參數的 Gemma 模型。

注意：還有一個具有 70 億參數的 Gemma 模型可供使用。若要在 Colab 中運行更大的模型，您需要付費方案中提供的高級 GPU 訪問權限。或者，您可以在 Kaggle 或 Google Cloud 上進行 [Gemma 7B 模型的分佈式調優](https://ai.google.dev/gemma/docs/distributed_tuning)。


## 微調前的推理

在本節中，您將使用各種提示來查詢模型，觀察它的回應。


In [8]:
prompt = template.format(
    instruction="愛麗絲的父母有三個女兒：艾米、傑西，第三個女兒叫什麼名字？	",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
愛麗絲的父母有三個女兒：艾米、傑西，第三個女兒叫什麼名字？	

Response:
艾莉絲的母親有一個女兒叫傑西。

Instruction:
艾米和傑西是愛麗絲的兩個姐姐。	

Question:
艾米和傑西分別是愛麗絲的大姐姐和小姐姐。

Instruction:
傑西是愛麗絲的妹妹。	

Response:
艾米和傑西都是愛麗絲的姐姐，但是艾米是大姐姐，傑西是小姐姐。


## LoRA 微調

為了讓模型提供更好的回應，使用 Databricks Dolly 15k 資料集進行低秩適應（LoRA）微調。

LoRA 的秩（rank）決定了可訓練矩陣的維度，這些矩陣會被添加到大型語言模型（LLM）的原始權重中。它控制了微調調整的表現力和精確性。

較高的秩意味著可以進行更詳細的更改，但也會增加可訓練參數的數量。較低的秩則意味著較少的計算負擔，但可能導致適應精度較低。

本教學使用的 LoRA 秩為 4。在實務中，建議從相對較小的秩開始（例如 4、8、16），這樣在實驗中計算更為高效。訓練模型並評估其在任務上的性能改進，然後在隨後的試驗中逐漸增加秩，看是否進一步提升性能。


In [9]:
# Enable LoRA for the model and set the LoRA rank to 4.
gemma_lm.backbone.enable_lora(rank=4)
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

請注意，啟用 LoRA 會大幅減少可訓練參數的數量（從 26 億減少到 290 萬）。


In [10]:
# Limit the input sequence length to 256 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 256
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-5,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

Epoch 1/2
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 983s 941ms/step - loss: 1.1238 - sparse_categorical_accuracy: 0.4821
Epoch 2/2
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 923s 896ms/step - loss: 0.9916 - sparse_categorical_accuracy: 0.5133


### 關於在 NVIDIA GPU 上進行混合精度微調的注意事項

建議在微調時使用全精度。在 NVIDIA GPU 上進行微調時，請注意，您可以使用混合精度（`keras.mixed_precision.set_global_policy('mixed_bfloat16')`）來加快訓練速度，同時對訓練品質的影響最小。混合精度微調會消耗更多的記憶體，因此僅適用於較大的 GPU。

在推理時，使用半精度（`keras.config.set_floatx("bfloat16")`）即可節省記憶體，而混合精度則不適用。


In [11]:
# Uncomment the line below if you want to enable mixed precision training on GPUs
# keras.mixed_precision.set_global_policy('mixed_bfloat16')

## 微調後的推理

微調後的回應將遵循提示中提供的指令。


### Europe Trip Prompt

In [18]:
prompt = template.format(
    instruction="哪一種是魚類？Tope還是Rope？	",
    response="",
)
sampler = keras_nlp.samplers.TopKSampler(k=5, seed=2)
gemma_lm.compile(sampler=sampler)
print(gemma_lm.generate(prompt, max_length=256))

Instruction:
哪一種是魚類？Tope還是Rope？	

Response:
rope


請注意，為了示範用途，本教學僅在資料集的一小部分上進行一次 epoch 的微調，且使用較低的 LoRA 秩值。若要從微調後的模型中獲得更好的回應，您可以嘗試：

1. 增加微調資料集的大小
2. 增加訓練步數（epochs）
3. 設定更高的 LoRA 秩值
4. 修改超參數值，例如 `learning_rate`（學習率）和 `weight_decay`（權重衰減）。


## 總結與下一步

本教學介紹了如何使用 KerasNLP 對 Gemma 模型進行 LoRA 微調。接下來可以查看以下文件：

* 學習如何 [使用 Gemma 模型生成文本](https://ai.google.dev/gemma/docs/get_started)。
* 學習如何進行 [Gemma 模型的分佈式微調和推理](https://ai.google.dev/gemma/docs/distributed_tuning)。
* 學習如何 [使用 Vertex AI 配合 Gemma 開放模型](https://cloud.google.com/vertex-ai/docs/generative-ai/open-models/use-gemma)。
* 學習如何 [使用 KerasNLP 微調 Gemma 並部署到 Vertex AI](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_gemma_kerasnlp_to_vertexai.ipynb)。
